In [64]:
using Pkg; Pkg.status("LoopVectorization")

      Status `D:\.julia\environments\v1.8\Project.toml`
  [bdcacae8] LoopVectorization v0.12.66 `https://github.com/JuliaSIMD/LoopVectorization.jl.git#master`


In [65]:
using LoopVectorization, BenchmarkTools
function findmin_turbo(x)
    indmin = 0
    minval = typemax(eltype(x))
    @turbo for i ∈ eachindex(x)
        newmin = x[i] < minval
        minval = newmin ? x[i] : minval
        indmin = newmin ? i    : indmin
    end
    minval, indmin
end

findmin_turbo (generic function with 1 method)

In [76]:
_min(x, y) = x < y ? x : y
myfindmin(f, domain) = mapfoldl(((k, v),) -> (f(v), k), _min , pairs(domain))
myfindmin(domain) = myfindmin(identity, domain)

myfindmin (generic function with 2 methods)

In [77]:
y = rand(1000);

@show findmin_turbo(y)
@show minimum(y)
@show myfindmin(y)
@show findmin(y)
@show argmin(y)

findmin_turbo(y) = (0.0013128019713591899, 942)
minimum(y) = 0.0013128019713591899
myfindmin(y) = (0.0013128019713591899, 942)
findmin(y) = (0.0013128019713591899, 942)
argmin(y) = 942


942

In [78]:
y = rand(1000);

@btime findmin_turbo($y)
@btime minimum($y)
@btime myfindmin($y)
@btime findmin($y)
@btime argmin($y)

  214.784 ns (0 allocations: 0 bytes)
  764.706 ns (0 allocations: 0 bytes)
  1.760 μs (0 allocations: 0 bytes)
  1.660 μs (0 allocations: 0 bytes)
  1.450 μs (0 allocations: 0 bytes)


638